In [64]:
import psycopg2
from sklearn import svm
import pandas.io.sql as psql
import psycopg2
import re
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import grid_search
import xgboost as xgb
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [65]:
import json
with open('/Users/krishna/MOOC/smart-city/login.json') as data_file:
    db = json.load(data_file)

conn = psycopg2.connect(database=db['dbname'], user=db['user'], password=db['password'], host=db['host'], port=db['port'])

In [66]:
dataframe = psql.frame_query("select idd, text, 2*alch_score +3 as alch_score_norm, local_score from twitter.tweets where (2*alch_score+3) - local_score < 1 and (2*alch_score+3 - local_score) > - 1 and alch_score != '0'  ;", conn)

In [67]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

In [68]:
dataframe['text']= dataframe['text'].apply(lambda x : processTweet(x))
dataframe['avg'] = dataframe.apply(lambda x : round((x['alch_score_norm'] + x['local_score'])/2) ,axis=1)

In [69]:
train_data, test_data = train_test_split(dataframe, test_size = 0.2)

In [70]:
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 1.0,
                             sublinear_tf=True,
                             use_idf=True)
train_vectors = vectorizer.fit_transform(train_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

In [71]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier 
clf1 = RandomForestClassifier(n_estimators=50)
forest = clf.fit(train_vectors, train_data['avg'])

In [72]:
clf2 = joblib.load('ModelObjects/modelsvm2804.pkl') 

In [73]:
from sklearn.naive_bayes import GaussianNB
clf3 =  GaussianNB()

In [74]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('rf', clf) ,('svm', clf2)], voting='hard')
eclf1 = eclf1.fit(train_vectors, train_data['avg'])
predict = forest.predict(test_vectors)
cm  = (confusion_matrix(test_data['avg'], predict))
print cm
print "Correctly Classified",cm.trace()
print "Total data point ",cm.sum()
#eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

[[ 5  1  1  0  0]
 [ 3 96 10  8  0]
 [ 1 18 33  7  0]
 [ 0 12 11 76  1]
 [ 0  0  0  2  3]]
Correctly Classified 213
Total data point  288
